In [5]:
import pandas as pd
import ast
import glob
import os

import numpy as np
import re
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler, StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
file_paths = glob.glob(os.path.join('../cleaned_data/', '*.xlsx'))

In [48]:
def Clean_third_party_insurance(df):
    df['Insurance_Validity_Period']=df['Insurance_Validity_Period'].replace({
        'Third Party insurance':'Third Party',
        'Zero Dep':'Zero Depreciation',
        np.nan:'Unknown',
        'Not Available': 'Unknown',
        '1': 'One Year',
        '2':'Two Years'
    })

    return df
    

In [49]:
def Clean_number_of_seats(df):
    
    df['Number_of_Seats'] = df['Number_of_Seats'].str.replace('Seats', '',regex= False).str.strip()
    df['Number_of_Seats'] = pd.to_numeric(df['Number_of_Seats'], errors='coerce')
    mean_value = df['Number_of_Seats'].mean()
    df['Number_of_Seats'] = df['Number_of_Seats'].fillna(mean_value)
    df['Number_of_Seats'] = df['Number_of_Seats'].astype(int)
    
    return df

In [56]:
def Clean_List_feature_columns(df):
    df['Comfort_and_Convenience'] = df['Comfort_and_Convenience'].astype(str)
    df['Comfort_and_Convenience'] = df['Comfort_and_Convenience'].fillna("unknown")
    df['Interior_Features'] = df['Interior_Features'].astype(str)
    df['Interior_Features'] = df['Interior_Features'].fillna("unknown")
    df['Exterior_Features'] = df['Exterior_Features'].astype(str)
    df['Exterior_Features'] = df['Exterior_Features'].fillna("unknown")
    df['Safety_Features'] = df['Safety_Features'].astype(str)
    df['Safety_Features'] = df['Safety_Features'].fillna("unknown")
    df['Entertainment_and_Communication'] = df['Entertainment_and_Communication'].astype(str)
    df['Entertainment_and_Communication'] = df['Entertainment_and_Communication'].fillna("unknown")


    return df

In [58]:
# Correct the loop to iterate over each tuple correctly
dfs = []
for file_path in file_paths:

    df = pd.read_excel(file_path)  # Load the data
    df = df.drop(columns=['Regional_Transport_Office', 'Original_Equipment_Manufacturer', 'Central_Variant_ID', 'Variant_Name'])
    df = Clean_third_party_insurance(df)
    df = Clean_number_of_seats(df)
    df = Clean_List_feature_columns(df)
    dfs.append(df)  # Append the DataFrame to the list
    

# Concatenate all DataFrames into one
combined_df = pd.concat(dfs, ignore_index=True)

In [59]:
combined_df.isnull().sum()

Insurance_Validity_Period             0
Fuel_Type                             0
Number_of_Seats                       0
Transmission_Type                     0
Top_Features                          0
Comfort_and_Convenience               0
Interior_Features                     0
Exterior_Features                     0
Safety_Features                       0
Entertainment_and_Communication       0
Mileage_(km/l)                      287
Engine_Capacity                       4
Maximum_Power                        60
Torque                               60
Wheel_Size                         2983
Battery_Type                          4
Kilometers_Driven                     0
Number_of_Owners                      0
Car_Model                             0
Model_Year                            0
Listed_Price                          0
Actual_Price                          0
city                                  0
dtype: int64

In [25]:
combined_df['Number_of_Seats'].value_counts()

Number_of_Seats
5 Seats     7354
7 Seats      728
6 Seats      103
4 Seats       93
8 Seats       62
9 Seats       11
2 Seats        7
10 Seats       5
Name: count, dtype: int64